In [ ]:
from pathlib import Path
from typing import Dict, List
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlwings as xl

In [ ]:
DATA_DIR = (
    Path("..")
    / ".."
    / ".."
    / "hfactory_magic_folders"
    / "plastic_cost_prediction"
    / "data"
)
MAIN_FILE = "BASF.xlsx"

In [ ]:
DATA_PATH = DATA_DIR / MAIN_FILE

In [ ]:
sheet_cols = {
    "Income Statement": [
        "Time",
        "Total Revenue",
        "Total Gross Profit",
        "Total Operating Profit/(Loss)",
    ],
    "Balance Sheet": [
        "Time",
        "Total Assets",
        "Total Liabilities",
        "Total Equity",
    ],
    "Cash Flow": [
        "Time",
        "Total Cash Flow from Operating Activities, Indirect",
        "Total Cash Flow from Investing Activities",
        "Total Cash Flow from Financing Activities",
    ],
}

In [ ]:
def BASF_preprocessing(
    data_path: str, sheet_cols: Dict[str, List[str]]
) -> Dict[str, pd.DataFrame]:
    """Given the path to the BASF file, preprocesses the data.

    Parameters
    ----------
    data_path: str
        Path to BASF file.
    sheet_cols: Dict[str, List[str]]
        Dict with the sheets to use and columns to keep.

    Returns
    -------
    BASF_dfs: Dict[str, pd.DataFrame]
        Dict containing the different processed dataframes(one per sheet).

    """
    # opening excel file so that excel formulas are calculated
    BASF_dfs = dict()
    app = xl.App(visible=False)
    book = app.books.open(data_path)
    book.save()
    app.kill()
    # obtaining dataframe for each sheet we select(key), with the columns we select(values)
    for key, values in sheet_cols.items():
        df = pd.read_excel(data_path, sheet_name=key, usecols="C:AV")
        df = df.T.iloc[:, 1:]
        df = df.rename(columns=df.iloc[0])
        df = df[10:]
        df = df.rename(columns={"Company: BASF (ETR: BAS)": "Time"})
        df = df.rename(columns=lambda x: str(x).strip())
        df = df[values]
        df = df.reset_index().drop("index", axis=1)
        BASF_dfs[key] = df
    return BASF_dfs

In [ ]:
dfs = BASF_preprocessing(DATA_PATH, sheet_cols)

In [ ]:
income_statement_df = dfs["Income Statement"]
balance_sheet_df = dfs["Balance Sheet"]
cash_flow_df = dfs["Cash Flow"]